In [20]:
import pandas as pd
from psycopg2 import connect
from sqlalchemy import create_engine

class Configuration:
    PG_HOST = 'business-intelligent-db.cw1neqwhyrda.eu-central-1.rds.amazonaws.com'
    PG_USER = 'postgres'
    PG_PWD = '52eXzbKbqmY2x45f'

PROCESSING_FILE_NAME = '1995.csv'
DB_NAME = 'airline'
TABLE_NAME = 'flight'

# class Configuration:
#     PG_HOST = 'localhost'
#     PG_USER = 'postgres'
#     PG_PWD = 'gherukdnbi6lrn4ieljb549ew9j4v98le5j9v48vl5ee'
    
engine = create_engine(f"postgresql://{Configuration.PG_USER}:{Configuration.PG_PWD}@{Configuration.PG_HOST}:5432/{DB_NAME}")
conn = engine.connect()
psy_conn = connect(host=Configuration.PG_HOST,
                   user=Configuration.PG_USER,
                   password=Configuration.PG_PWD,
                   database="airline",
                   port=5432)


In [2]:
import os
import psycopg2
# Not correct way to copy to pg db table
def copy_from_file(conn, table):
    # save the dataframe on disk as a csv file, load
    # the csv file and use copy_from() to copy it to the table
    tmp_df = f"./result/{PROCESSING_FILE_NAME}"
    # df.to_csv(tmp_df, header=True, index=False)
    f = open(tmp_df, 'r')
    cursor = conn.cursor()
    try:
        cursor.copy_from(f, table, sep=',')
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        # os.remove(tmp_df)
        print("Error: %s" %error)
        conn.rollback()
        conn.close()
        return 1
    print("copy_from_file() done")
    cursor.close()
    # os.remove(tmp_df)

In [3]:
# Not use anymore
# connect to our postgres DB
conn = connect(host=Configuration.PG_HOST,
                user=Configuration.PG_USER,
                password=Configuration.PG_PWD,
                database="airline",
                port=5432)
copy_from_file(conn, 'flight')

Error: invalid input syntax for type integer: "year"
CONTEXT:  COPY flight, line 1, column year: "year"



1

## Load data from csv (corrected way)

In [21]:
processed_flight_df = f"./result/{PROCESSING_FILE_NAME}"
flight_df = pd.read_csv(processed_flight_df)
flight_df.dtypes

year                     int64
month                    int64
day_of_month             int64
day_of_week              int64
dep_time                object
crs_dep_time            object
arr_time                object
crs_arr_time            object
unique_carrier          object
flight_num               int64
tail_num                object
actual_elapsed_time    float64
crs_elapsed_time       float64
air_time               float64
arr_delay              float64
dep_delay              float64
origin                  object
dest                    object
distance               float64
taxi_in                  int64
taxi_out                 int64
cancelled                int64
diverted                 int64
dtype: object

In [5]:
def convert_df_to_db_data_type(df):
    df['cancelled'] = df['cancelled'].astype(bool)
    df['diverted'] = df['diverted'].astype(bool)

In [23]:
# convert pandas df to db record data type
convert_df_to_db_data_type(flight_df)
flight_df.dtypes
flight_df.head()

,year,month,day_of_month,day_of_week,dep_time,crs_dep_time,arr_time,crs_arr_time,unique_carrier,flight_num,...,air_time,arr_delay,dep_delay,origin,dest,distance,taxi_in,taxi_out,cancelled,diverted
0,1995,1,6,5,1995-01-06 06:57:00,1995-01-06 06:45:00,1995-01-06 09:52:00,1995-01-06 09:37:00,UA,482,...,83.0,15.0,12.0,ORD,PHL,678.0,7,25,False,False
1,1995,1,7,6,1995-01-07 06:48:00,1995-01-07 06:45:00,1995-01-07 09:38:00,1995-01-07 09:37:00,UA,482,...,88.0,1.0,3.0,ORD,PHL,678.0,5,17,False,False
2,1995,1,8,7,1995-01-08 06:49:00,1995-01-08 06:45:00,1995-01-08 09:32:00,1995-01-08 09:37:00,UA,482,...,83.0,-5.0,4.0,ORD,PHL,678.0,3,17,False,False
3,1995,1,9,1,1995-01-09 06:45:00,1995-01-09 06:45:00,1995-01-09 09:28:00,1995-01-09 09:37:00,UA,482,...,84.0,-9.0,0.0,ORD,PHL,678.0,3,16,False,False
4,1995,1,10,2,1995-01-10 06:45:00,1995-01-10 06:45:00,1995-01-10 09:31:00,1995-01-10 09:37:00,UA,482,...,82.0,-6.0,0.0,ORD,PHL,678.0,6,18,False,False


In [24]:
# Save df to db
flight_df.to_sql(TABLE_NAME, engine, if_exists='append', index=False)

psy_conn.autocommit = True
cursor = psy_conn.cursor()
  
sql1 = 'SELECT count(*) from flight;'
cursor.execute(sql1)
for i in cursor.fetchall():
    print(i)

# conn.commit()
conn.close()

(46063768,)
